In [119]:
#to find whether the '-' is an operator or represents <0. and simplize the formula.
# e.g -3/-2 -> 3/2 ; 3/-7 -> -3/7
def change_operator(part):
    
    index = 1
    count = 0
    while index < len(part)-1:
        if part[index] in '/*' and part[index+1] is '-':
            count += 1
        index += 1
    
    replaced = part[1:].replace('-','')
    if count%2 == 0:
        return part[0]+replaced
    else:
        if part[0] == '+':
            return '-' +replaced
        else:
            return '+' +replaced

In [120]:
#To split the input line into lines which are start with '+' or '-'.
def split_by_symbol(line):
    
    symbol = '+-*/'
    if line[0].isdigit():
        my_line = '+' + line
    else:
        my_line = line
    
    index = 0
    stop_symbol = []
    lines = []
    result = []
    
    while index < len(my_line):
        if (my_line[index] in {'+','-'}) and (my_line[index-1] not in symbol):
            stop_symbol.append(index)
        index += 1
    
    for i in range(len(stop_symbol)):
        try:
            lines.append(my_line[stop_symbol[i]:stop_symbol[i+1]])
        except IndexError:
            lines.append(my_line[stop_symbol[i]:len(my_line)])
            
    for l in lines:
        if '/-' or '*-' in l:
            result.append(change_operator(l))
            
    
    return result

In [51]:
#read from position index
def readNumber(line, index):
    number = 0
    while index < len(line) and line[index].isdigit():
        number = number * 10 + int(line[index])
        index += 1
    
    if index < len(line) and line[index] == '.':
        index += 1
        keta = 0.1
    while index < len(line) and line[index].isdigit():
        number += int(line[index]) * keta
        keta /= 10
        index += 1
    token = {'type': 'NUMBER', 'number': number}
    return token, index

In [145]:
def read_operator(type, index):
    if type == '+':
        token = {'type': 'PLUS'}
        return token, index + 1
    if type == '-':
        token = {'type':'MINUS'}
        return token, index + 1    
    if type == '*':
        token = {'type': 'MUL'}
        return token, index + 1
    if type == '/':
        token = {'type': 'DIV'}
        return token, index + 1

In [146]:
def tokenize(line):
  tokens = []
  index = 0
  while index < len(line):
    if line[index].isdigit():
      (token, index) = readNumber(line, index)
    elif read_operator(line[index], index):
      (token, index) = readPlus(line, index)
    else:
      print('Invalid character found: ' + line[index])
      break
    tokens.append(token)
  return tokens

In [147]:
# calculate the 'x' and '/'
def evaluate_mul_and_div(tokens):

    length = len(tokens)
    index = 2 # tokens[0] would be '+' or '-' based on my 'split by symbol'.
    if length == 2:
        return tokens[1]['number']
    else:
        answer = tokens[index-1]['number']
        while index<length:
            if tokens[index]['type'] == 'MUL':
                answer *= tokens[index+1]['number']
            if tokens[index]['type'] == 'DIV':
                answer /= tokens[index+1]['number']
            index += 2
    return answer

In [148]:
#calculate the '+' and '-'
def evaluate_plus_and_minus(line):
  
    lines = split_by_symbol(line)

    answer = 0
    for l in lines:
        token = tokenize(l)
        if token[0]['type'] == 'PLUS':
            answer += evaluate_mul_and_div(token)
        elif token[0]['type'] == 'MINUS':
            answer -= evaluate_mul_and_div(token)
        else:
            print('Invalid syntax')
            exit(1)
    
    return answer

In [149]:
#calculate with ()

def evaluate_with_parentheses(line):
        
    if line.find('(') == -1:
        return evaluate_plus_and_minus(line)
    
    length = len(line)
    index = 0
    right_parentheses = 0
    left_parentheses = 0
    while index < length:
        if line[index] == '(':
            left_parentheses = index
        if line[index] == ')':
            right_parentheses = index
            
        if right_parentheses:
            result = evaluate_plus_and_minus(line[left_parentheses+1:right_parentheses])
            new_line = line[:left_parentheses] + str(result) + line[right_parentheses+1:]
            
            return find_parentheses(new_line)
        
        index += 1
        

        

In [ ]:
def test(line):
  actualAnswer = evaluate_with_parentheses(line)
  expectedAnswer = eval(line)
  if abs(actualAnswer - expectedAnswer) < 1e-8:
    print("PASS! (%s = %f)" % (line, expectedAnswer))
  else:
    print("FAIL! (%s should be %f but was %f)" % (line, expectedAnswer, actualAnswer))


# Add more tests to this function :)
def runTest():
  print("==== Test started! ====")
  test("1+2")
  test("1.0+2.1-3")
  test("0+0")
  test("-3/(1-3)")
  test("((1+2)*(3+4)/(7-(11-2)))*(21+19)")
  print("==== Test finished! ====\n")

runTest()

while True:
  print('> ', end="")
  line = input()
  answer = evaluate_with_parentheses(line)
  print("answer = %f\n" % answer)

==== Test started! ====
PASS! (1+2 = 3.000000)
FAIL! (1.0+2.1-3 should be 0.100000 but was 6.100000)
PASS! (0+0 = 0.000000)
FAIL! (-3/(1-3) should be 1.500000 but was 3.000000)
FAIL! (((1+2)*(3+4)/(7-(11-2)))*(21+19) should be -420.000000 but was 3.000000)
==== Test finished! ====

> 